## Load Earth System Science Data (ESSD) datasets

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import sys

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

# import sys
# sys.path.append('../src/')

In [ ]:
# import re
import pandas as pd
import numpy as np

import openpyxl

import pint
import pint_pandas
import iam_units
from openscm_units import unit_registry
pint_pandas.PintType.ureg = unit_registry
ureg = unit_registry

In [ ]:
import osc_ingest_trino as osc
import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

Create an S3 resource for the bucket holding source data

In [ ]:
import boto3
s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
# Create an S3 client.  We will user later when we write out data and metadata
s3_trino = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [ ]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'

In [ ]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

Load ESSD data file using pandas *read_excel* and using *ingest_uuid* as the global UUID for this ingestion

In [ ]:
# Much better to use disk than memory for large datasets (at least as far as notebook resources are concerned)!
s3_essd_file = bObj = bucket.Object('ESSD/essd_ghg_data.xlsx')
essd_tmpfile = f'/tmp/essd-data.xlsx'
s3_essd_file.download_file(essd_tmpfile)
timestamp = bObj.last_modified.isoformat()

# It takes ~90 seconds (!) to load nearly 600K rows of data
df = pd.read_excel(essd_tmpfile, sheet_name='data',
                   converters={'year': lambda x: pd.to_datetime(x, format='%Y')},
                   dtype={'gwp100_ar5':'int32', 'value':'float64'},
                   engine='openpyxl')

In [ ]:
# Put year at the end to make for more friendly partitioning
year_index = df.columns.get_loc('year')
essd_data_df = df[df.columns[:year_index].to_list()
                  + df.columns[year_index+1:].to_list()
                  + [df.columns[year_index]]]
essd_data_df = essd_data_df.convert_dtypes()

display(essd_data_df.columns)

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
essd_content_df = pd.read_excel(essd_tmpfile, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Dataset',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Data description']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

essd_metadata_df = pd.read_excel(essd_tmpfile, sheet_name='metadata')
custom_meta_fields = { d['Variable']: {k:v for k,v in d.items() if k!='Variable' and v==v} for d in essd_metadata_df.to_dict('records') }

The schemaname for this table is `essd`.  Dunno if it's a good idea or simply redundant to prefix the tablename with `essd_`

In [ ]:
essd_data_df.info(verbose=True)

In [ ]:
ingest_table = 'ghg_data'
columnschema = osc.create_table_schema_pairs(essd_data_df,
                                             typemap={"datetime64[ns]":"timestamp(6)"})

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
format = 'ORC',
partitioning = array['year']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
essd_data_df.to_sql(ingest_table,
                    con=engine, schema=ingest_schema, if_exists='append',
                    index=False,
                    method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))
del(essd_data_df)

Verify that we can restore data and metadata

Grab Sector, Region, 100yr GWPs, and GH4_gwps from one of the two main ESSD data tables (they are the same in both).

In [ ]:
essd_sectors_df = pd.read_excel(essd_tmpfile, sheet_name='sector_classification', engine='openpyxl').convert_dtypes()

display(essd_sectors_df.columns)

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Sector Classification',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Sectors']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

ingest_table = 'sectors'
columnschema = osc.create_table_schema_pairs(essd_sectors_df,
                                             typemap={"datetime64[ns]":"timestamp(6)"})

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
format = 'ORC'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
essd_sectors_df.to_sql(ingest_table,
                       con=engine, schema=ingest_schema, if_exists='append',
                       index=False,
                       method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))

In [ ]:
essd_regions_df = pd.read_excel(essd_tmpfile, sheet_name='region_classification', engine='openpyxl').convert_dtypes()

display(essd_regions_df.columns)

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Region Classification',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

ingest_table = 'regions'
columnschema = osc.create_table_schema_pairs(essd_regions_df,
                                             typemap={"datetime64[ns]":"timestamp(6)"})

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
format = 'ORC'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
essd_regions_df.to_sql(ingest_table,
                       con=engine, schema=ingest_schema, if_exists='append',
                       index=False,
                       method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))

Now deal with the gas species.  We'll annotate our dataframe with PINT units

In [ ]:
essd_gwp_df = pd.read_excel(essd_tmpfile, sheet_name='100_yr_gwps', dtype={'gwp_ar5':'int32'}, engine='openpyxl')

display(essd_gwp_df.columns)

In [ ]:
ureg.define("CH4_Combustion = nan CH4")
ureg.define("CH4_Fugitive = nan CH4")
ureg.define("CH4_Process = nan CH4")

ch4_dict = {
    'CH4 Biogenic': 'CH4',
    'CH4 Fossil (Combustion)': 'CH4_Combustion',
    'CH4 Fossil (Fugitive)': 'CH4_Fugitive',
    'CH4 Fossil (Process)': 'CH4_Process'
}

def convert_gas_to_pint_species(s):
    """For a series S of gas species, return the species name known to pint"""
    return [ch4_dict[g] if g in ch4_dict else g.replace('c-', 'C').replace('-','') for g in s.tolist()]

essd_gwp_df = essd_gwp_df.assign(units=lambda x: convert_gas_to_pint_species(x.gas)).convert_dtypes()

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Gas Species',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

ingest_table = 'gwp_100yr'
columnschema = osc.create_table_schema_pairs(essd_gwp_df,
                                             typemap={"datetime64[ns]":"timestamp(6)"})

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
format = 'ORC'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
essd_gwp_df.to_sql(ingest_table,
                   con=engine, schema=ingest_schema, if_exists='append',
                   index=False,
                   method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))

In [ ]:
essd_ch4_df = pd.read_excel(essd_tmpfile, sheet_name='CH4_gwps', dtype={'gwp_ar5':'int32', 'subsector':'str'}, engine='openpyxl').convert_dtypes()

display(essd_ch4_df.columns)

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) CH4 GWPs',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

ingest_table = 'ch4_gwp'
columnschema = osc.create_table_schema_pairs(essd_ch4_df,
                                             typemap={"datetime64[ns]":"timestamp(6)"})

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
format = 'ORC'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
essd_ch4_df.to_sql(ingest_table,
                   con=engine, schema=ingest_schema, if_exists='append',
                   index=False,
                   method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))

In [ ]:
s3_gwp100_file = bObj = bucket.Object('ESSD/essd_ghg_data_gwp100.xlsx')
essd_gwp100_tmpfile = f'/tmp/essd-gwp100.xlsx'
s3_gwp100_file.download_file(essd_gwp100_tmpfile)
timestamp = bObj.last_modified.isoformat()

# This takes about 30sec to execute
df = pd.read_excel(essd_gwp100_tmpfile, sheet_name='data',
                   converters={'year': lambda x: pd.to_datetime(x, format='%Y')},
                   dtype={'gwp100_ar5':'int32', 'value':'float64'},
                   engine='openpyxl').convert_dtypes()

# Put year at the end to make for more friendly partitioning
year_index = df.columns.get_loc('year')
essd_gwp100_df = df[df.columns[:year_index].to_list()
                    + df.columns[year_index+1:].to_list()
                    + [df.columns[year_index]]]

display(essd_gwp100_df.columns)

In [ ]:
essd_gwp100_content_df = pd.read_excel(essd_gwp100_tmpfile, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) GWP100 Dataset',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Data description']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

essd_gwp100_metadata_df = pd.read_excel(essd_gwp100_tmpfile, sheet_name='metadata')
custom_meta_fields = { d['Variable']: {k:v for k,v in d.items() if k!='Variable' and v==v} for d in essd_metadata_df.to_dict('records') }

In [ ]:
custom_meta_content

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
ingest_table = 'gwp100_data'
columnschema = osc.create_table_schema_pairs(essd_gwp100_df,
                                             typemap={"datetime64[ns]":"timestamp(6)"})

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
format = 'ORC',
partitioning = array['year']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
essd_gwp100_df.to_sql(ingest_table,
                      con=engine, schema=ingest_schema, if_exists='append',
                      index=False,
                      method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))
del(essd_gwp100_df)

In [ ]:
# This table is not so large that we need to load it via the filesystem

import io

bObj = bucket.Object('ESSD/essd_lulucf_data.xlsx')
lulucf_bytes = io.BytesIO(bucket.Object(bObj.key).get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

df = pd.read_excel(lulucf_bytes, sheet_name='data', converters={'year': lambda x: pd.to_datetime(x, format='%Y')}, engine='openpyxl').convert_dtypes()

# Put year at the end to make for more friendly partitioning
year_index = df.columns.get_loc('year')
lulucf_df = df[df.columns[:year_index].to_list()
               + df.columns[year_index+1:].to_list()
               + [df.columns[year_index]]]

display(lulucf_df.columns)

In [ ]:
lulucf_content_df = pd.read_excel(lulucf_bytes, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Land Use Change and Forestry Dataset',
    'author': lulucf_content_df[['Author']].squeeze(),
    'contact': lulucf_content_df[['Contact']].squeeze(),
    'description': lulucf_content_df[['Info']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = { 'Units': { k:'tCO2' for k in ['blue', 'houghton', 'oscar', 'mean'] }}

In [ ]:
custom_meta_content

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
ingest_table = 'lulucf'
columnschema = osc.create_table_schema_pairs(lulucf_df,
                                             typemap={"datetime64[ns]":"timestamp(6)"})

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
format = 'ORC',
partitioning = array['year']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
lulucf_df.to_sql(ingest_table,
                 con=engine, schema=ingest_schema, if_exists='append',
                 index=False,
                 method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*